In [1]:
import cv2
import numpy as np
from poly_point_isect import isect_segments
from scipy.spatial.distance import pdist, euclidean, squareform

In [2]:
img = cv2.imread('imgs/brd.11.jpg')

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
height, width = img.shape[:2]
center = width/2, height/2

### Blur the image

In [3]:
kernel_size = 9
sigma = 120
# blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)
blur_gray = cv2.bilateralFilter(gray,6,sigma,sigma)

### Find edges

In [4]:
low_threshold = 80
high_threshold = 120
edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

### Draw lines

In [5]:
line_image = np.copy(img) * 0 

lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 40, np.array([]), 25, 46)

line_segments = []
r = 0.1
for line in lines:
    for x1,y1,x2,y2 in line:
        x3,y3 = x1+(x1-x2)*r, y1+(y1-y2)*r
        x4,y4 = x2+(x2-x1)*r, y2+(y2-y1)*r
        line_segments.append(((x3, y3), (x4, y4)))
        cv2.line(line_image,(int(x3),int(y3)),(int(x4),int(y4)),(255,0,0),3)

line_image = cv2.addWeighted(img, 0.8, line_image, 0.7, 0)

### Find intersections

In [6]:
isects = isect_segments(line_segments)
distances = squareform(pdist(isects, 'euclidean'))

# Use the distances matrix to group agreeing intersections together 
# -> agreeing intersections are within 20px of each other
# -> the groups are stored by their index in the 'isects' list

tol = 20
visited = []
groups = []
for i in range(len(isects)):
    if i not in visited:
        neighbours = []
        for j in range(len(isects)):
            if distances[i][j] < tol:
                neighbours.append(j)
                visited.append(j)
        groups.append(neighbours)

### Calculate positions

In [7]:
class position():
    def __init__(self, pos):
        self.pos = pos
        self.ofset = euclidean(center, pos)
        self.x = pos[0]
        self.y = pos[1]

In [8]:
positions = []
for group in groups:
    points = [isects[i] for i in group]
    avg = np.mean(points, axis=0)
    positions.append(position(avg))

positions = sorted(positions, key=lambda p: p.ofset)
positions = positions[:24]
for p in positions:
    x, y = p.pos
    cv2.circle(line_image, (int(x), int(y)), 2, (0, 255, 0), 4) 

## Image Skewing

In [9]:
pts1 = np.float32([p.pos for p in positions[-4:]])
pts2 = np.float32([[0,0],[500,0],[0,500],[500,500]])
M = cv2.getPerspectiveTransform(pts1, pts2)
skew = cv2.warpPerspective(img,M,(500,500))

In [10]:
small_img = cv2.resize(skew, (7,7), interpolation=cv2.INTER_NEAREST)

## OPEN Image

In [ ]:
cv2.imshow('image', line_image)
cv2.waitKey(0)
cv2.destroyAllWindows()